---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [78]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [79]:
# Use this dictionary to map state names to two letter acronyms

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [80]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni_town = pd.read_csv('university_towns.txt', header=None, sep="\t", names=['Raw Data'])
    uni_town.loc[uni_town['Raw Data'].str.contains('\[edit'), 'State'] = uni_town['Raw Data'].str.replace('\[edit\]', '').str.strip()
    pattern = '\(.+'
    uni_town.loc[~uni_town['Raw Data'].str.contains('\[edit'), 'RegionName'] = uni_town['Raw Data'].str.replace(pattern, '').str.strip()
    
    uni_town['State'].fillna(method='ffill', inplace=True)
    #uni_town = uni_town[~uni_town['RegionName'].isnull()]
    uni_town.dropna(inplace=True)
    uni_town = uni_town[['State', 'RegionName']]
    
    return uni_town  #.shape

#get_list_of_university_towns()

In [81]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # may need to define recession start as quarter before the recession start
    gdp_df = pd.read_excel('gdplev.xls',skiprows=7, usecols=[4,6], names=['quarter', 'gdp_chained'])
    gdp_df = gdp_df[gdp_df['quarter'] >= '2000q1']
    gdp_df['gdp_delta'] = gdp_df['gdp_chained'].diff()
    gdp_df['Recession'] = ((gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(1) < 0) | (gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(-1) < 0)) # flag recession
    gdp_df = gdp_df[gdp_df['Recession'] == True]
    gdp_df = gdp_df.sort_values(by='quarter')
    return gdp_df.iloc[0]['quarter'] #.sort_values(by='quarter', inplace=True)

get_recession_start()

'2008q3'

In [82]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp_df = pd.read_excel('gdplev.xls',skiprows=7, usecols=[4,6], names=['quarter', 'gdp_chained'])
    gdp_df = gdp_df[gdp_df['quarter'] >= '2000q1']
    gdp_df['gdp_delta'] = gdp_df['gdp_chained'].diff()
    gdp_df['Recession'] = ((gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(1) < 0) | (gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(-1) < 0)) # flag recession
    gdp_df2 = gdp_df[gdp_df['Recession'] == True]
    gdp_df2 = gdp_df2.sort_values(by='quarter', ascending=False).index[0]
    return  gdp_df.loc[gdp_df2+2][0] #gdp_df.loc[gdp_df2] #[0]['quarter'] #.sort_values(by='quarter', inplace=True)   

#get_recession_end()

In [83]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp_df = pd.read_excel('gdplev.xls',skiprows=7, usecols=[4,6], names=['quarter', 'gdp_chained'])
    gdp_df = gdp_df[gdp_df['quarter'] >= '2000q1']
    gdp_df['gdp_delta'] = gdp_df['gdp_chained'].diff()
    gdp_df['Recession'] = ((gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(1) < 0) | (gdp_df['gdp_delta'] < 0) & (gdp_df['gdp_delta'].shift(-1) < 0)) # flag recession
    gdp_df = gdp_df[gdp_df['Recession'] == True]
    #gdp_df = gdp_df.loc(gdp_df['gdp_delta'].idxmin())
    #gdp_df = gdp_df[gdp_df.index == gdp_df['gdp_delta'].idxmin()]
    return  gdp_df.loc[gdp_df['gdp_chained'].idxmin()]['quarter'] #gdp_df.iloc[0]['quarter']

#get_recession_bottom()

In [84]:
def determine_qtr(month):
    
    qtr_dict = {'01': 'q1', '02': 'q1', '03':'q1', 
               '04': 'q2', '05':'q2', '06':'q2', 
                '07': 'q3', '08':'q3', '09':'q3', 
                '10':'q4', '11':'q4', '12':'q4'}
    
    return qtr_dict[month]

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    ''' 
    
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_df.replace({'State': states}, inplace = True)  # map states with long name
    housing_df.set_index(['State', 'RegionName'], inplace = True)
    housing_df.drop(['RegionID','Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)    
    #housing_df = housing_df.groupby(lambda x: '{}{}'.format(x[:4], qtr_dict[x[5:7]]), axis=1).sum()   
    housing_df = housing_df.groupby(lambda x: '{}{}'.format(x[:4],determine_qtr(x[5:7])), axis=1).mean()
    housing_df = housing_df.iloc[:, 15:]
    return housing_df #.loc['NY']

#convert_housing_data_to_quarters()

In [88]:
import scipy.stats as stats

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    recession_start =  get_recession_start()
    recession_bottom = get_recession_bottom()
    df_housing = convert_housing_data_to_quarters()
    #print(recession_start)
    #df_housing['price_ratio'] = df_housing['2008q2']/df_housing[recession_bottom]
    df_housing['price_ratio'] = df_housing[recession_start]/df_housing[recession_bottom]
    df_housing = df_housing[['price_ratio', recession_start, recession_bottom]] # may require quarter before recession start
    df_unit_towns = get_list_of_university_towns()
    df_unit_towns['One'] = '1'
    df_unit_towns.set_index(["State", "RegionName"], inplace = True)
  
    df_housing = pd.merge(df_housing, df_unit_towns, how='left', left_index=True, right_index=True)
    df_non_universities = df_housing[df_housing['One'].isnull()]
    df_universities   = df_housing[~df_housing['One'].isnull()]    
   
    test_stats =  stats.ttest_ind(df_universities['price_ratio'], df_non_universities['price_ratio'], nan_policy='omit')
    p_val = test_stats[1]
    better = "university town" if df_universities['price_ratio'].mean() < df_non_universities['price_ratio'].mean() else "non-university town"                   
    different = True if p_val < 0.01 else False
    
    return  (different, p_val, better)#test_stats #test_stats #df_universities #test_stats

#run_ttest()

(True, 0.0054964273536330264, 'university town')